In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats


class XDistGen(object):
    def __init__(self, dist, dim=2):
        self.dim = dim
        self.dist = dist
        
    def __call__(self, samples):
        return self.dist.rvs(size=(samples, self.dim))

    def pdf(self, x):
        return self.dist.pdf(x).prod(axis=1)


class YGen1(object):
    def __init__(self, dim=3):
        self.dim = dim

    def __call__(self, x):
        y = np.zeros((x.shape[0], self.dim))
        y[:, 0] = 1/(1 + np.exp(-x[:, 0] + 1)) / 2 + 0.5
        y[:, 1] = (1 - 1/(1 + np.exp(-x[:, 1] - 1)))
        y[:, 2] = (1 - 1/(1 + np.exp(-x[:, 1] - 3)))
        return y
    
class YGen2(object):
    def __init__(self, dim=3):
        self.dim = dim

    def __call__(self, x):
        y = YGen1(dim=self.dim)(x)
        #y[:, 2] = 1/(1 + np.exp(-(x[:, 0] + x[:, 1])))
        y[:, 2] = 1/(1 + np.exp(-x[:, 0] + 2))
        return y

In [ ]:
samples = 100000
x_gen = XDistGen(stats.norm, dim=2)
x = x_gen(samples)
x_prob = x_gen.pdf(x)

fig = plt.figure()
ax = fig.add_subplot(111, projection = '3d')
ax.scatter(x[:,0], x[:,1], x_prob, c=x_prob, cmap='viridis')

In [ ]:
y1 = YGen1(dim=3)(x)
y2 = YGen2(dim=3)(x)

In [ ]:
fig = plt.figure(figsize=(22, 5))
ax = fig.add_subplot(1, 4, 1, projection='3d')
ax.scatter(x[:,0], x[:,1], y1[:,0], c=y1[:,0], cmap='viridis')
ax = fig.add_subplot(1, 4, 2, projection='3d')
ax.scatter(x[:,0], x[:,1], y1[:,1], c=y1[:,1], cmap='viridis')
ax = fig.add_subplot(1, 4, 3, projection='3d')
ax.scatter(x[:,0], x[:,1], y1[:,2], c=y1[:,2], cmap='viridis')
ax = fig.add_subplot(1, 4, 4, projection='3d')
ax.scatter(x[:,0], x[:,1], y2[:,2], c=y2[:,2], cmap='viridis')

In [ ]:
from pathlib import Path
import os
import sys
sys.path.append(os.path.dirname(Path.cwd()))
from src.frank_wolfe import frank_wolfe, select_top_k_np, macro_sqrt_tp_C, macro_f1_C


def get_order(y_proba, classifiers, classifier_weights, seed=0):
    if seed is not None:
        np.random.seed(seed)

    ni = y_proba.shape[0]
    result = np.zeros(y_proba.shape)
    for i in range(ni):
        c = np.random.choice(classifiers.shape[0], p=classifier_weights)
        G = classifiers[c]
        eta_i = y_proba[i]
        u = (eta_i * (G[:,0] - G[:,1] - G[:,2] + G[:,3])) + G[:,1] - G[:,3]
        result[i] = u

    return result


def get_fw_order01(y, utlity_func):
    classifiers, classifiers_weights, meta = frank_wolfe(y, y, utlity_func, max_iters=100, k=1)
    y_pred = get_order(y, classifiers, classifiers_weights, seed=0)
    order = y_pred[:,0] > y_pred[:,1]
    return order, y_pred


In [ ]:
order1, y_pred1 = get_fw_order01(y1, macro_f1_C)
order2, y_pred2 = get_fw_order01(y2, macro_f1_C)

In [ ]:
y_pred1

In [ ]:
y_pred2

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(16.5, 5))
diff = order1 != order2
axes[0].scatter(x[:,0], x[:,1], c=order1, cmap='viridis')
axes[1].scatter(x[:,0], x[:,1], c=order2, cmap='viridis')
axes[2].scatter(x[:,0], x[:,1], c=diff, cmap='viridis')